In [30]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import time
from tqdm import tqdm_notebook as tq
import warnings
import pickle as pkl
warnings.filterwarnings("ignore")
import string
import sys
from nltk.corpus import stopwords
plt.ion()

In [ ]:
reviews = pd.read_csv("model/master_reviews.csv")

In [ ]:
reviews.head()

In [4]:
reviews.shape

(6685899, 11)

In [5]:
reviews_mod = reviews.dropna()

In [6]:
reviews_mod.shape

(6685820, 11)

In [2]:
class SentimentAnalysis:
    def __init__(self):
        
        self.map_word = {}
        self.map_index = {}
        self.x_y_pairs = {}
        self.pairs_map = {}
        
#    def word_embed(self)
    
#     def word2vec(self):
#         self.window_size = 2
#         self.epochs = 100
#         self.embeddings = 10
#         self.vocab_size = length(sla.pairs_map[self.window_size].keys())
        
#         weight1 = torch.randn(self.embeddings,self.vocab_size)
#         weight2 = torch.randn(self.vocab_size,self.embeddings)
        
#         for epoch in epochs:
#             for center,context in 
    
    def NGRAM(self):
        pass

In [48]:

class TrainSelfEmbedding:
    
    def __init__(self):
        self.window_size = 2
        self.epochs = 100
        self.embeddings = 10
        self.epoch_loss = 0
        
    def train(self,pairs_map,epochs = 100,sample_size=10**4):
        
        self.vocab_size = len(sla.map_word.keys())
        
        print(self.vocab_size)
        
        
        weight1 = torch.randn(self.embeddings,self.vocab_size).float()
        weight2 = torch.randn(self.vocab_size,self.embeddings).float()
        
        print(sys.getsizeof(weight1.storage()),weight1.element_size()*weight1.nelement()/10**6)
        print(sys.getsizeof(weight2.storage()),weight2.element_size()*weight2.nelement()/10**6)
        
        #total_loss = 0
        
        self.epochs = epochs
        
        for i in tq(range(self.epochs)):
            
            total_loss = 0
            
            
            sampled_pairs = random.sample(pairs_map[self.window_size],sample_size)
            
            for i in range(sample_size):

                center,target = sampled_pairs[i]
                inpt = torch.zeros(self.vocab_size).float()
                inpt[center] = 1.0
                z1 = torch.matmul(weight1,inpt)
                z2 = torch.matmul(weight2,z1)

                log_output = torch.log_softmax(z2,dim=0)

#                 print(inpt.size())
#                 print(weight1.size())
#                 print(z1.size())


#                 print(weight2.size())
#                 print(z2.size())
#                 print(log_output.size())

                loss = torch.nn.functional.nll_loss(log_output.view(1,-1),torch.tensor([target]))

                total_loss += loss

            
            

In [3]:
def tokenize_vocab(map_word,map_index,x):
    
    idx = len(map_word.keys())
    ans = []
    #print(type(x),x)
    for word in x.split():
        if word not in map_word:
            idx+=1
            map_word[word] = idx
            map_index[idx] = word
        ans.append(idx)
            
    return ans


def build_context_center(x,pairs_map,window = 2):
    
    if window not in pairs_map:
        pairs_map[window] = []
        
    #x = x.split()
    for i in range(len(x)):
        
        for j in range(1,window+1):
            
            if (i-j)>0:
                pairs_map[window].append([x[i],x[i-j]])
                
            if (i+j)<len(x):
                pairs_map[window].append([x[i],x[i+j]])
                
                

In [9]:
%%time
sla = SentimentAnalysis()
reviews_mod['tokenized'] = reviews_mod.text.apply(lambda x: tokenize_vocab(sla.map_word,sla.map_index,x))

CPU times: user 2min 13s, sys: 3.12 s, total: 2min 16s
Wall time: 2min 16s


In [10]:
reviews_mod.text_length.max()

4755

In [11]:
%%time
_ = reviews_mod.iloc[:,].tokenized.apply(lambda x:build_context_center(x,sla.pairs_map))

CPU times: user 19min 8s, sys: 1min 13s, total: 20min 21s
Wall time: 20min 19s


In [12]:
sla.pairs_map[2][:10]

[[1, 2],
 [1, 3],
 [2, 3],
 [2, 4],
 [3, 2],
 [3, 4],
 [3, 5],
 [4, 3],
 [4, 5],
 [4, 2]]

In [13]:
%%time
import pickle as pkl
with open("model/map.pkl",'wb') as file:
    pkl.dump(sla.pairs_map,file)

CPU times: user 14min 46s, sys: 1min 36s, total: 16min 22s
Wall time: 18min 53s


In [14]:
%%time
with open("model/map_word.pkl",'wb') as file:
    pkl.dump(sla.map_word,file)

CPU times: user 1.53 s, sys: 241 ms, total: 1.77 s
Wall time: 2.14 s


In [15]:
%%time

with open("model/map_index.pkl",'wb') as file:
    pkl.dump(sla.map_index,file)

CPU times: user 1.45 s, sys: 225 ms, total: 1.67 s
Wall time: 1.97 s


### Get pre-cleaned data

In [4]:
%%time

sla = SentimentAnalysis()
with open("model/map.pkl",'rb') as file:
    sla.pairs_map = pkl.load(file)
    
print("pairs map import complete**********")  

with open("model/map_word.pkl",'rb') as file:
    sla.map_word = pkl.load(file)
    
print("map word import complete*************")
    
with open("model/map_index.pkl",'rb') as file:
    sla.map_index = pkl.load(file)
    
print("map index import complete*************")
    

pairs map import complete**********
map word import complete*************
map index import complete*************
CPU times: user 15min 37s, sys: 4min 19s, total: 19min 56s
Wall time: 22min


In [45]:
self_embedding = TrainSelfEmbedding()

In [46]:
%%time
self_embedding.train(sla.pairs_map,epochs= 50, sample_size= 1000)

3020286
120811512 120.81144
120811512 120.81144


torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.S

torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.S

torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.S

torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.S

...

torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.S

torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size(

torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.S

torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.Size([10])
torch.Size([3020286, 10])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([3020286])
torch.Size([10, 3020286])
torch.S

KeyboardInterrupt: 